[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1b-sg2SfnlSLUReyNDVnqtfpYFI_jf3i0?usp=sharing)

# Техническая часть

In [1]:
!wget -q https://www.dropbox.com/s/43l702z5a5i2w8j/gazeta_train.txt
!wget -q https://www.dropbox.com/s/k2egt3sug0hb185/gazeta_val.txt
!wget -q https://www.dropbox.com/s/3gki5n5djs9w0v6/gazeta_test.txt

In [2]:
# Extractive
!pip install -q sentence-transformers
!pip install -q razdel

# Abstractive
!pip install -q transformers
!pip install -q sentencepiece
!pip install -q pytorch_lightning
!pip install -q rouge

     |████████████████████████████████| 71kB 6.5MB/s 
     |████████████████████████████████| 1.3MB 20.1MB/s 
     |████████████████████████████████| 2.9MB 35.5MB/s 
     |████████████████████████████████| 1.1MB 55.0MB/s 
     |████████████████████████████████| 890kB 40.7MB/s 
     |████████████████████████████████| 563kB 14.1MB/s 
     |████████████████████████████████| 276kB 55.5MB/s 
     |████████████████████████████████| 829kB 49.6MB/s 
     |████████████████████████████████| 92kB 13.0MB/s 


In [3]:
!git clone https://github.com/DanilDmitriev1999/extractive_summarization

Cloning into 'extractive_summarization'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 118 (delta 55), reused 81 (delta 28), pack-reused 0
Receiving objects: 100% (118/118), 469.96 KiB | 26.11 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [4]:
from itertools import combinations
import networkx as nx
import json
import random
from tqdm import tqdm

from sentence_transformers import SentenceTransformer, util

from scipy.sparse.csgraph import connected_components

import numpy as np
from scipy.linalg import norm

from nltk.translate.bleu_score import corpus_bleu
import nltk
import razdel

from pprint import pprint

Весь код лежит в [моём](https://github.com/DanilDmitriev1999/extractive_summarization) репозитории

In [5]:
from extractive_summarization.utils import *
from extractive_summarization.text_rank import *
from extractive_summarization.lex_rank import *

# Введение

**Text Summarization** - это процесс автоматического создания сжатой версии данного текста, предоставляющей полезную информацию пользователю.

Есть 2 вида суммаризации текста:
1.  Abstractive summarization - генерируем маленький текст, который описывает "своими словами" исходный текст.
2.  Extractive summarization - выбираем из исходного текста предложения, которые лучше всего передают смысл текста. 

Чисто extractive summarization часто дают лучшие результаты по сравнению с автоматическими Abstractive summarization. Это связано с тем, что проблемы абстрактивного обобщения, такие как семантическое представление, вывод и генерация естественного языка, относительно сложнее по сравнению с подходом, основанным на данных, таким как извлечение предложений. На самом деле, по-настоящему Abstractive summarization еще не достигло сегодня зрелой стадии. 

Но сегодня с каждым годом Abstractive summarization становится лучше и лучше, например такие модели, как [T5](https://arxiv.org/pdf/1910.10683.pdf), [Pegasus](https://ai.googleblog.com/2020/06/pegasus-state-of-art-model-for.html), [BART](https://arxiv.org/abs/1910.13461) показывают уже хорошие резулультаты, после fine-tuning на своих данных.

## Метрики

**BLUE** - метрика которая принимает значение [0, 1]. BLUE, это что-то вроде точности угадывания n-gram из правильного summary.

**ROUGE** (Recall-Oriented Understudy for Gisting Evaluation). Есть несколько типов ROUGE:
1. *ROUGE-1*: сравнивается каждое слово
2. *ROUGE-2*: сравниваются би-граммы
3. *ROUGE-L*: статистика на основе самой длинной общей подпоследовательности (LCS). Задача самой длинной общей подпоследовательности естественным образом учитывает сходство структуры на уровне предложения и автоматически определяет самую длинную совместную последовательность n-граммов.

## План ноутбука



1.   Загрузка данных
2.   TextRank
3.   LexRank
4.   zero-shot mT5 
5.   fine-tuning mT5



# Данные

In [6]:
train_records = read_gazeta_records("gazeta_train.txt", 52400)
val_records = read_gazeta_records("gazeta_val.txt", 5265)
test_records = read_gazeta_records("gazeta_test.txt", 5770)

100%|██████████| 5770/5770 [00:00<00:00, 19387.90it/s]


In [ ]:
pprint(train_records[219], width=130)

{'date': '2018-08-22 08:01:56',
 'summary': 'Перед контрольными прокатами, которые пройдут 9 сентября в Москве, российские фигуристы примут участие в сборе в '
            'Новогорске. Любители этого вида спорта ждут не только возвращения спортсменов на лед, но и встречи серебряного '
            'призера Олимпийских игр 2018 года Евгении Медведевой с бывшим тренером Этери Тутберидзе.',
 'text': 'До старта нового сезона в фигурном катании остается все меньше времени. Первый этап Гран-при пройдет в американском '
         'городе Эверетт с 19 по 21 октября. Перед этим представители сборной России примут участие в контрольных прокатах, '
         'которые пройдут 9 сентября в Москве на арене «Мегаспорт». Еще раньше спортсмены примут участие в сборе в Новогорске. '
         'Генеральный директор Федерации фигурного катания на коньках России (ФФККР) Александр Коган заявил, что в Новогорск '
         'прибудет двукратный серебряный призер Олимпийских игр 2018 года в Пхенчхане Евгения Медвед

# Extractive summarization 

## TextRank

### Теория

TextRank - это алгоритм, основанный на PageRank, который часто используется для извлечения ключевых слов и суммаризации текста. 

PageRank (PR) - это алгоритм, используемый для расчета веса веб-страниц. Мы можем взять все веб-страницы в виде большого ориентированного графа. На этом графике узел является веб-страницей. Если веб-страница A имеет ссылку на веб-страницу B, она может быть представлена ​​в виде направленного ребра от A до B.

### Пробуем на практике

In [ ]:
model_name = 'distilbert-multilingual-nli-stsb-quora-ranking'
model = TextRank(model_name)

100%|██████████| 501M/501M [00:22<00:00, 22.0MB/s]


In [ ]:
model.get_summary(train_records[14])


Count: 1
Полный текст:
('В пятницу, 27 мая, в столичном регионе давление повысится до 749 мм рт. ст. Как пояснили «Газете.Ru» в центре «Фобос» , из-за этого солнце сильнее '
 'прогреет воздух: днем ожидается +22…+27°С (ночью +9…+14°С). Атмосфера станет неустойчивой, а это, в свою очередь, вызовет короткие дожди и грозы — '
 'преимущественно днем. Из-за духоты, которая всегда возникает перед грозой, жара будет переноситься еще хуже, так что всем страдающим от '
 'сердечно-сосудистых заболеваний или болезней органов дыхания стоит быть втройне осторожнее. Ветер ночью переменных направлений, слабый, днем '
 'восточный, северо-восточный, 3–8 м/с, при грозе — порывистый. Последний уикенд весны поддержит погодный тренд: синоптическая ситуация в Москве '
 'существенно не изменится. Оба дня переменная облачность, днем местами кратковременный дождь, не исключена гроза. Давление высокое — 751 мм рт. '
 'ст., ветер в субботу южный, в воскресенье — северо-восточный, оба дня скорость одинаковая, 3–

### Test TextRank

In [ ]:
model.get_summary(train_records[4321])


Count: 1
Полный текст:
('Ситуацию в Белоруссии рассмотрят на заседании Совета ООН по правам человека. С таким предложением выступил Евросоюз. «Одним из главных приоритетов '
 'ЕС в перспективе сессии Совета ООН по правам человека, которая состоится в июне, будет ситуация с правами человека в Белоруссии, которая '
 'продолжает вызывать горячую обеспокоенность», – сказано в специальном постановлении Совета ЕС , принятом в связи с подготовкой регулярной сессии '
 'Совета ООН. Документ был принят в понедельник в Брюсселе министрами иностранных дел стран ЕС. В нем также говорится, что Евросоюз считает '
 'приоритетным на ближайшей сессии обсудить и ситуацию с правами человека в Сирии, где нарушается международное право. Также Совет ЕС обращает '
 'внимание Совета ООН на критическое положение с правами человека в КНДР. Евросоюз совместно с Японией предложит на сессии создать независимый '
 'механизм расследования ситуации с обеспечением прав человека в Северной Корее. Наконец, ЕС намерен ак

In [ ]:
model.get_summary(train_records[2900])

100%|██████████| 501M/501M [00:21<00:00, 23.3MB/s]



Count: 1
Полный текст:
('«Подтверждаю, что губернатор прилетел», — подтвердил ТАСС информацию о возвращении губернатора неназванный источник, близкий к Тулеев у. '
 'Губернатор был доставлен в Кемеровскую область чартерным рейсом. Его встречал автомобиль скорой помощи и замгубернатора по медицине Валерий Цой. '
 'Самочувствие Амана Тулеева называют хорошим. «Состояние губернатора стабильное, он чувствует себя значительно лучше. Сейчас он будет проходить '
 'реабилитационный курс в регионе, будет включаться в работу, в том числе планирует проводить рабочие встречи с подчиненными», — добавил собеседник '
 'агентства. Официально Тулеев находился в отпуске начиная с 22 мая. Изначально глава Кемеровской области взял отпуск на 10 дней, но после срок '
 'неоднократно продлевался. Все это время обязанности главы региона исполнял первый заместитель губернатора Владимир Чернов. Операция на '
 'позвоночнике губернатора длилась 9 часов и была проведена в клинике, расположенной в Германии. Врачи п

Есть поддержка своих текстов.

In [ ]:
model.get_summary(train_records[2900]['text'])

Полный текст:
('«Подтверждаю, что губернатор прилетел», — подтвердил ТАСС информацию о возвращении губернатора неназванный источник, близкий к Тулеев у. '
 'Губернатор был доставлен в Кемеровскую область чартерным рейсом. Его встречал автомобиль скорой помощи и замгубернатора по медицине Валерий Цой. '
 'Самочувствие Амана Тулеева называют хорошим. «Состояние губернатора стабильное, он чувствует себя значительно лучше. Сейчас он будет проходить '
 'реабилитационный курс в регионе, будет включаться в работу, в том числе планирует проводить рабочие встречи с подчиненными», — добавил собеседник '
 'агентства. Официально Тулеев находился в отпуске начиная с 22 мая. Изначально глава Кемеровской области взял отпуск на 10 дней, но после срок '
 'неоднократно продлевался. Все это время обязанности главы региона исполнял первый заместитель губернатора Владимир Чернов. Операция на '
 'позвоночнике губернатора длилась 9 часов и была проведена в клинике, расположенной в Германии. Врачи приняли реш

## LexRank

### Теория

LexRank используется для вычисления важности предложения на основе концепции центральности собственного вектора в представлении предложений с помощью графа.



В LexRank у нас есть матрица связности, основанная на косинусном сходстве внутри предложения, которая используется в качестве матрицы смежности графа представления предложений. Выбирается центроидное предложение, которое работает как среднее для всех других предложений в документе. Затем предложения ранжируются по их сходству.



### Пробуем на практике

In [ ]:
model_name = 'distilbert-multilingual-nli-stsb-quora-ranking'
lexrank_model = LexRank(model_name)
lexrank_model.get_summary(train_records[4321], n_sentences_summary=3)

100%|██████████| 501M/501M [00:17<00:00, 28.9MB/s]



Count: 1
Полный текст:
('Ситуацию в Белоруссии рассмотрят на заседании Совета ООН по правам человека. С таким предложением выступил Евросоюз. «Одним из главных приоритетов '
 'ЕС в перспективе сессии Совета ООН по правам человека, которая состоится в июне, будет ситуация с правами человека в Белоруссии, которая '
 'продолжает вызывать горячую обеспокоенность», – сказано в специальном постановлении Совета ЕС , принятом в связи с подготовкой регулярной сессии '
 'Совета ООН. Документ был принят в понедельник в Брюсселе министрами иностранных дел стран ЕС. В нем также говорится, что Евросоюз считает '
 'приоритетным на ближайшей сессии обсудить и ситуацию с правами человека в Сирии, где нарушается международное право. Также Совет ЕС обращает '
 'внимание Совета ООН на критическое положение с правами человека в КНДР. Евросоюз совместно с Японией предложит на сессии создать независимый '
 'механизм расследования ситуации с обеспечением прав человека в Северной Корее. Наконец, ЕС намерен ак

### Test LexRank

In [ ]:
lexrank_model.get_summary(train_records[4321], n_sentences_summary=3, only_blue=True)

0.3568631835520593

In [ ]:
lexrank_model.get_summary(train_records[14]['text'])

Полный текст:
('В пятницу, 27 мая, в столичном регионе давление повысится до 749 мм рт. ст. Как пояснили «Газете.Ru» в центре «Фобос» , из-за этого солнце сильнее '
 'прогреет воздух: днем ожидается +22…+27°С (ночью +9…+14°С). Атмосфера станет неустойчивой, а это, в свою очередь, вызовет короткие дожди и грозы — '
 'преимущественно днем. Из-за духоты, которая всегда возникает перед грозой, жара будет переноситься еще хуже, так что всем страдающим от '
 'сердечно-сосудистых заболеваний или болезней органов дыхания стоит быть втройне осторожнее. Ветер ночью переменных направлений, слабый, днем '
 'восточный, северо-восточный, 3–8 м/с, при грозе — порывистый. Последний уикенд весны поддержит погодный тренд: синоптическая ситуация в Москве '
 'существенно не изменится. Оба дня переменная облачность, днем местами кратковременный дождь, не исключена гроза. Давление высокое — 751 мм рт. '
 'ст., ветер в субботу южный, в воскресенье — северо-восточный, оба дня скорость одинаковая, 3–8 м/с. Тем



---



# Abstractive summarization mT5

![Текст ссылки](https://miro.medium.com/max/2003/1*D0J1gNQf8vrrUpKeyD8wPA.png)


"Text-to-Text Transfer Transformer" **T5** - это представитель семейства Transformer, в отличие от BERT, T5 полноценная seq2seq модель. 
Очевидная новая идея, лежащая в основе этой работы, заключается в том, что это модель преобразования текста в текст: во время обучения модели предлагается создать новый текст в качестве выходных данных, даже для обучающих задач, которые обычно моделируются как задачи классификации и регрессии с большим количеством более простые виды вывода. Однако эта идея, похоже, была выбрана из соображений инженерного удобства, и нет никаких доказательств того, что она необходима для полученных результатов. Вместо этого, что делает эту работу успешной (и впечатляющей), так это то, что авторы взяли многие из лучших идей из ряда недавних работ по НЛП и проделали чрезвычайно хорошую работу по тщательному тестированию и уточнению каждой идеи по мере ее добавления. Это дало как очень хорошо настроенную модель, так и большое понимание тонкоструктурных проектных решений, которые используются для обучения языку большой нейронной сети общего назначения.

mT5 - это мультиязычный аналог T5, кроме обучающего набора данных ничего нового добавлено не было.

In [7]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import torch.nn as nn
import torch.nn.functional as F


import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.loggers import WandbLogger

import matplotlib.pyplot as plt

from rouge import Rouge

import torch
import json
import sentencepiece
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config, AdamW, get_linear_schedule_with_warmup

In [8]:
model = T5ForConditionalGeneration.from_pretrained('google/mt5-small')
tokenizer = T5Tokenizer.from_pretrained('google/mt5-small')

Some weights of the model checkpoint at google/mt5-small were not used when initializing T5ForConditionalGeneration: ['encoder.block.0.layer.1.DenseReluDense.wi_0.weight', 'encoder.block.0.layer.1.DenseReluDense.wi_1.weight', 'encoder.block.1.layer.1.DenseReluDense.wi_0.weight', 'encoder.block.1.layer.1.DenseReluDense.wi_1.weight', 'encoder.block.2.layer.1.DenseReluDense.wi_0.weight', 'encoder.block.2.layer.1.DenseReluDense.wi_1.weight', 'encoder.block.3.layer.1.DenseReluDense.wi_0.weight', 'encoder.block.3.layer.1.DenseReluDense.wi_1.weight', 'encoder.block.4.layer.1.DenseReluDense.wi_0.weight', 'encoder.block.4.layer.1.DenseReluDense.wi_1.weight', 'encoder.block.5.layer.1.DenseReluDense.wi_0.weight', 'encoder.block.5.layer.1.DenseReluDense.wi_1.weight', 'encoder.block.6.layer.1.DenseReluDense.wi_0.weight', 'encoder.block.6.layer.1.DenseReluDense.wi_1.weight', 'encoder.block.7.layer.1.DenseReluDense.wi_0.weight', 'encoder.block.7.layer.1.DenseReluDense.wi_1.weight', 'decoder.block.0.l

## Zero-shot

Попробуем сначала использовать модель "из коробки" не дообучая на своих данных.

In [ ]:
device = torch.device('cpu')

In [ ]:
pprint(train_records[4321], width=150)

{'date': '2013-02-19 13:14:02',
 'summary': 'Противостояние Белоруссии и Запада переходит на принципиально новый уровень. Если до сих пор с репрессиями и несменяемой властью '
            'Александра Лукашенко боролись исключительно европейцы при некоторой поддержке США, то теперь поведение официального Минска будет '
            'рассмотрено на специальной сессии Совета ООН по правам человека, которая состоится в июне. С инициативой выступил ЕС. Режиму Лукашенко '
            'это может грозить серьезными проблемами: даже если ООН не примет против Минска международных санкций, Евросоюз ужесточит свои.',
 'text': 'Ситуацию в Белоруссии рассмотрят на заседании Совета ООН по правам человека. С таким предложением выступил Евросоюз. «Одним из главных '
         'приоритетов ЕС в перспективе сессии Совета ООН по правам человека, которая состоится в июне, будет ситуация с правами человека в '
         'Белоруссии, которая продолжает вызывать горячую обеспокоенность», – сказано в специальном 

In [ ]:
tokenized_text = tokenizer.encode(train_records[4321]['text'], return_tensors="pt").to(device)

In [ ]:
summary_ids = model.generate(tokenized_text,
                                    num_beams=20,
                                    no_repeat_ngram_size=2,
                                    min_length=200,
                                    max_length=400,
                                    early_stopping=True)

output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

pprint(output, width=150)

('<extra_id_0>. Кроме того, ЕС намерен активно поддержать продление мандата спецдокладчика на ближайшей сессии обсудить ситуацию с правами человека '
 'в Сирии и в результате чего они будут принимать участие в заседании Совета ООН по правам человека. Также также говорится, что в случае обострения '
 'конфликта с Россией нет никакого отношения к этому вопросу. По мнению эксперта «Белой газете.Ru» <extra_id_10>и <extra_id_25>. '
 '<extra_id_49>.жамит.com.uanicznych санкций в отношении белорусского режима. <extra_id_9>жами. Новости о политзаключенных. Минск, 18 февраля 2013 '
 'г. по состоянию на несколько дней со дня официального визита в Варшаву с докладом. Павел Харашти. Фото: БЕЛОРУССКОЕ ПРАВО ЛЮДИ В РОССИЙСКОЙ '
 'ГОСУДАРСТВЕННЫЙ ЦЕНТР ЕВРОПЕЙСКОГО СОВРЕМЕННО.')


Такое себе, саммари вообще не про то.